# Sandbox
Notebook to test code before real implementation

In [1]:
################ ONLY PYTHON 2 #########################
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations, chain, product
from collections import defaultdict
import argparse
import time
import os
import tensorflow as tf
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn import metrics
import pandas as pd
import psutil
import pickle
from decagon.deep.optimizer import DecagonOptimizer
from decagon.deep.model import DecagonModel
from decagon.deep.minibatch import EdgeMinibatchIterator
from decagon.utility import rank_metrics, preprocessing

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import time
import os
import psutil
import pickle
from pybdm import BDM
from pybdm.utils import decompose_dataset
from pybdm.partitions import PartitionIgnore
from joblib import Parallel, delayed
from joblib import parallel_backend
from data.algorithms import PerturbationExperiment, NodePerturbationExperiment
import math
import datetime as dt
from itertools import product

# Guardado de predictions en diccionario

In [ ]:
opti = 'data/data_structures/intento_optimizer'
with open(opti, 'rb') as f:
    opt = pickle.load(f)

In [ ]:
print(opt.keys())
len(opt[1,1])

In [ ]:
H = np.random.rand(3,4)
A = np.random.rand(4,4)-0.15
A = np.abs(np.round((A + A.T)/2))
W = np.random.rand(6,3)

In [ ]:
A

In [ ]:
H_2 = np.dot(np.dot(W,H),A)
H_2

In [ ]:
W_2 = np.random.rand(9,6)

In [ ]:
embeddings = np.dot(np.dot(W_2,H_2),A)
embeddings

# TWOSIDES

In [2]:
DF = pd.read_csv('../Thesis_datasets/TWOSIDES/small.csv',sep=',',usecols=[0,1,2,3,4,5])
DEC = pd.read_csv('data/original_data/bio-decagon-combo.csv',sep=',')

In [3]:
DEC.head()

,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name
0,CID000002173,CID000003345,C0151714,hypermagnesemia
1,CID000002173,CID000003345,C0035344,retinopathy of prematurity
2,CID000002173,CID000003345,C0004144,atelectasis
3,CID000002173,CID000003345,C0002063,alkalosis
4,CID000002173,CID000003345,C0004604,Back Ache


In [4]:
DF.head(10)

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name
0,10355,Temazepam,136411,sildenafil,10003239,Arthralgia
1,1808,Bumetanide,7824,Oxytocin,10003239,Arthralgia
2,221147,POLYETHYLENE GLYCOL 3350,5521,Hydroxychloroquine,10003239,Arthralgia
3,10324,Tamoxifen,8640,Prednisone,10012735,Diarrhoea
4,10355,Temazepam,136411,sildenafil,10012735,Diarrhoea
5,1808,Bumetanide,7824,Oxytocin,10012735,Diarrhoea
6,161,Acetaminophen,1546438,glycopyrronium,10012735,Diarrhoea
7,7781,Oxazepam,9863,Sodium Chloride,10012735,Diarrhoea
8,221147,POLYETHYLENE GLYCOL 3350,5521,Hydroxychloroquine,10012735,Diarrhoea
9,10355,Temazepam,136411,sildenafil,10019211,Headache


### Transform Dictionaries Side effects
`meddra.tsv` is a database that contains names of side effects in both wanted ids

In [5]:
map_se = pd.read_csv('../Thesis_datasets/SIDER/meddra.tsv', sep = '\t'\
                     ,header=None).rename(columns={0:'UMLS',1:'kind',2:'MedDRA',3:'name'})
cui = pd.unique(map_se['UMLS'].values)
meddra = pd.unique(map_se['MedDRA'].values)
ses = pd.unique(map_se['name'].values)
print('Total',len(map_se))
print('CUI',len(cui))
print('MedDRA',len(meddra))
print('Side Effects',len(ses))
map_se.head(5)

Total 95912
CUI 49561
MedDRA 74359
Side Effects 75604


,UMLS,kind,MedDRA,name
0,C0000727,LT,10000647,Acute abdomen
1,C0000727,PT,10000647,Acute abdomen
2,C0000727,LT,10042784,Syndrome abdominal acute
3,C0000727,LT,10000096,Abdominal syndrome acute
4,C0000729,LT,10000057,Abdominal cramps


In [6]:
# Dictionary that translates UMLS CUIs to MedDRA IDs
umls2meddra = defaultdict(set)
for se in map_se.index:
    umls2meddra[map_se.loc[se,'UMLS']] = map_se.loc[se,'MedDRA']
# Dictionary that translates MedDRA IDs to UMLS CUIs
meddra2umls = defaultdict(set)
for se in map_se.index:
    meddra2umls[map_se.loc[se,'MedDRA']] = map_se.loc[se,'UMLS']
# Dictionary that translates UMLS CUIs to name
names = map_se[map_se['kind'].str.match('PT',na=False)].reset_index(drop=True)
print(len(names.index))
UMLS2names = {}
for se in names.index:
    UMLS2names[names.loc[se,'UMLS']] = names.loc[se,'name']
# Verify numbers
print(len(umls2meddra))
print(len(meddra2umls))
print(len(UMLS2names))

49561
74359


In [7]:
# Dictionary that translates UMLS CUIs to name
names = map_se[map_se['kind'].str.match('PT',na=False)].reset_index(drop=True)
print(len(names.index))
UMLS2names = {}
for se in names.index:
    UMLS2names[names.loc[se,'UMLS']] = names.loc[se,'name']
print(len(UMLS2names))

20307
19973


### Replacing 

In [8]:
ids = []
for i in DF['condition_meddra_id']:
    ids.append(meddra2umls[i])
DF['Condition'] = ids
DF = DF.drop(columns=['condition_meddra_id'])
name_list = []
for i in DF['Condition']:
    name_list.append(UMLS2names[i])
DF['Condition_name'] = name_list
DF = DF.drop(columns=['condition_concept_name'])
DF.head(10)

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,Condition,Condition_name
0,10355,Temazepam,136411,sildenafil,C0003862,Arthralgia
1,1808,Bumetanide,7824,Oxytocin,C0003862,Arthralgia
2,221147,POLYETHYLENE GLYCOL 3350,5521,Hydroxychloroquine,C0003862,Arthralgia
3,10324,Tamoxifen,8640,Prednisone,C0011991,Diarrhoea
4,10355,Temazepam,136411,sildenafil,C0011991,Diarrhoea
5,1808,Bumetanide,7824,Oxytocin,C0011991,Diarrhoea
6,161,Acetaminophen,1546438,glycopyrronium,C0011991,Diarrhoea
7,7781,Oxazepam,9863,Sodium Chloride,C0011991,Diarrhoea
8,221147,POLYETHYLENE GLYCOL 3350,5521,Hydroxychloroquine,C0011991,Diarrhoea
9,10355,Temazepam,136411,sildenafil,C0018681,Headache


In [ ]:
map_drug = pd.read_csv('../Thesis_datasets/SIDER/drug_names.tsv',sep = '\t',header=None)

In [ ]:
map_drug.head()

## Testing fixed unigram candidate sampler found in optimization

In [ ]:
sess = tf.Session()
a = [7,0,20,8,33,9]
A = tf.reshape(tf.constant(a,dtype=tf.int64),[6,1])
sampled_ids, true_expected_count, sampled_expected_count = tf.nn.fixed_unigram_candidate_sampler(
   true_classes = A,
   num_true = 1,
   num_sampled = 20,
   unique = False,
   range_max = np.shape(a)[0],
   unigrams = [ 10, 10, 10, 10, 50, 10 ]
)
sample = tf.gather( A, sampled_ids )
print(sess.run( true_expected_count ))
print(sess.run( sampled_ids ))
print(sess.run( sampled_expected_count ))
print(sess.run( sample ))

In [ ]:
in_file = 'data/data_structures/DECAGON/DECAGON_real_affinities_genes_16814_drugs_276_se_7'
with open(in_file, 'rb') as f:
    DS = pickle.load(f)
    for key in DS.keys():
        globals()[key]=DS[key]
        print(key,"Imported successfully")

In [ ]:
adj_mats_orig[1,0][0].todense().sum()

In [ ]:
val_metrics[0,:,:]

In [ ]:
%pylab inline

In [ ]:
pd.read_csv('../Thesis_datasets/DrugBank/drugbank_all_full_database.xml/full database.xml')